<a href="https://colab.research.google.com/github/hamed-jamali-software/mp32face/blob/main/wav2lipHigh%20Definition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super Wav2Lip

This Colab project is based on [Wav2Lip-GFPGAN](https://github.com/ajay-sainy/Wav2Lip-GFPGAN), but updates the requirements.txt (to function properly) and updates Colab file for ease of use.

## 1. Installation

Run this block to install the necessary dependencies.

In [1]:
!git clone https://github.com/indianajson/wav2lip-HD.git
basePath = "/content/wav2lip-HD"
%cd {basePath}

wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'

!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O {wav2lipPath}'/checkpoints/wav2lip_gan.pth'
#!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O {wav2lipPath}'/checkpoints/wav2lip.pth'

!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

!pip install -r requirements.txt
!pip install -U librosa==0.8.1 # The process will fail without downgrading librosa
!mkdir inputs

!cd $gfpganFolderName && python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'

%cd {basePath}

from IPython.display import clear_output
clear_output()

print("Installation complete.")

Installation complete.


In [4]:
!pip install "numpy<1.24"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xgboost 2.1.1 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
gfpgan 1.3.4 requires basicsr>=1.3.4.0, which is not installed.
gfpgan 1.3.4 requires facexlib>=0.2.3, which is not installed.
gfpgan 1.3.4 requires lmdb, which is not installed.
albucore 0.0.13 requires numpy<2,>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.12 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.1.

## 2. Synchronize Video and Speech

In [5]:
#@markdown Adjust the parameters below then run the code block to synthesize the speech onto your video file:

import os
outputPath = basePath+'/outputs'
inputAudio = '5.mp3' #@param{type:"string"}
inputAudioPath = basePath + '/inputs/' + inputAudio
inputVideo = 'short.mp4' #@param{type:"string"}
inputVideoPath = basePath + '/inputs/'+inputVideo
lipSyncedOutputPath = basePath + '/outputs/result.mp4'
model = "wav2lip_gan" #@param ["wav2lip", "wav2lip_gan"] {type:"string"}


if not os.path.exists(outputPath):
  os.makedirs(outputPath)

from IPython.display import clear_output
clear_output()

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/{model}.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}



#print("Video synthesis complete.")

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 92
Extracting raw audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libs

## 3. Boost the Resolution of the Synthesized Video



In [6]:
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)


!cd $gfpganFolderName && \
  python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 2 --only_center_face --bg_upsampler None

import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

if not os.path.exists(restoredFramesPath):
  os.makedirs(restoredFramesPath)

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

#Get FPS of original video for writer
inputVideoPath = outputPath+'/result.mp4'
vidcap = cv2.VideoCapture(inputVideoPath)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("The video is "+str(fps)+" FPS.")

batch = 0
batchSize = 1300
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end, end="\r")
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
  out = cv2.VideoWriter(processedVideoOutputPath+'/output_'+str(batch).zfill(4)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

from IPython.display import clear_output
clear_output()

print("Video upscaling complete.")

Video upscaling complete.


## 4. Clear Cached Files

Run this block once you've downloaded your final video file. This will empty /inputs and /outputs, so you can start again, fresh.


In [7]:
%cd /content/wav2lip-HD/

#@markdown Choose whether to remove both inputs and outputs, or just one of the two. You may want to preserve inputs if you are only changing one of the two inputs.

removeInputs = True #@param {type:"boolean"}
removeOutputs = True #@param {type:"boolean"}

if removeInputs == True:
  %rm inputs/*
if removeOutputs == True:
  %rm outputs/frames/*
  %rm outputs/restored_imgs/*
  %rm outputs/*


from IPython.display import clear_output
clear_output()

print("Cleared cached files.")


Cleared cached files.
